# Using LSTM to predict NVDA prices

Using LSTM:
- Maintains an internal state, allowing it to have memory of previous inputs

1.	Handling Sequential Data: LSTM (Long Short-Term Memory) networks are specifically designed to handle and predict sequences of data, making them well-suited for time series forecasting like stock prices.
2.	Capturing Long-Term Dependencies: LSTM networks can remember information for long periods, thanks to their ability to maintain a memory cell that updates and forgets information over time. This is crucial for stock price prediction, where historical data can influence future values.

In [10]:
# importing libraries

import pandas as pd # data visualization
import numpy as np # number manipulation
from sklearn.preprocessing import MinMaxScaler # helps to normalize data
from keras.models import Sequential # allows you to build a NN in a sequential stack
from keras.layers import LSTM, Dense # to define the neural network being used + hidden layers

# note that need to pip install tensorflow - keras is built on tensorflow

In [11]:
# Load and preprocess data
data = pd.read_csv('./NVDA.csv') # use pandas to read the given csv file
data = data[['Date', 'Close']]
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)